In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

In [33]:
MODEL_ARCHITECTURE = {
    'conv_block_filters': [16, 32, 64],
    'conv_block_kernel_sizes': [3, 3, 3],
    'conv_block_strides': [2, 2, 2],
    'conv_block_dropout_rates': [0.2, 0.2, 0.2],
    'dense_block_units': [32],
    'dense_block_dropout_rates': [0.5]
}

N_EPOCHS = 50
N_TRIALS = 50
BATCH_SIZE = 512

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [ ]:
x = []
y = []

ds_1 = tfds.load("stl10", split="train", as_supervised=True, shuffle_files=False)
for x_, y_ in ds_1.as_numpy_iterator():
    x.append(x_)
    y.append(y_)
    
ds_2 = tfds.load("stl10", split="test", as_supervised=True, shuffle_files=False)
for x_, y_ in ds_2.as_numpy_iterator():
    x.append(x_)
    y.append(y_)

x = np.array(x, dtype=np.float32) / 255
y = np.array(y, dtype=np.float32)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.95, random_state=42, stratify=y)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]
train_size, len(x_val), len(x_test), train_size + len(x_val) + len(x_test)

In [ ]:
x_train_sorted = x_train[np.argsort(y_train)]
y_train_sorted = y_train[np.argsort(y_train)]
_, counts = np.unique(y_train_sorted, return_counts=True)

In [ ]:
model = ClassifierModel(output_shape=n_classes, **MODEL_ARCHITECTURE)
model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
model(x_train[0:1])
model.load_weights("../models/default_model_weights_stl10.weights.h5")
model_weights = model.get_weights()

In [31]:
def validate_name(value_type: str, order_type: utils.OrderType, name: str):
    assert name.startswith(f"{value_type}_{order_type}_")

In [ ]:
model_fit_1_params = {
    "epochs": N_EPOCHS,
    "batch_size": BATCH_SIZE,
    "shuffle": True,
    "verbose": 0,
}

model_fit_2_params = {
    "epochs": 500,
    "batch_size": BATCH_SIZE,
    "shuffle": True,
    "verbose": 0,
}

In [32]:
def run_experiments(value_type: str, order_type: utils.OrderType, name: str):
    validate_name(value_type, order_type, name)
    acc, re_mi, re_ma, pr_mi, pr_ma, f1_mi, f1_ma = [], [], [], [], [], [], []
    best_model_weights = model.get_weights()
    
    if value_type == "edges":
        samples_values = utils.calculate_values_edges(x_train_sorted, blur=True)
        samples_proba = utils.normalize_values_per_group(samples_values, counts)
    
    for _ in tqdm(range(N_TRIALS)):
        model.set_weights(model_weights)
    
        for i in range(N_EPOCHS):
            n_samples = int(np.tanh(4 * (i + 1) / N_EPOCHS) * train_size)

            if value_type == "losses":
                samples_values = utils.calculate_values_losses(model, x_train_sorted, y_train_sorted, batch_size=BATCH_SIZE)
                samples_proba = utils.normalize_values_per_group(samples_values, counts)

            samples_ids = utils.chose_samples(n_samples, samples_proba, order_type)
    
            model.fit(x_train_sorted[samples_ids], y_train_sorted[samples_ids], **model_fit_1_params)
    
        model.fit(
            x_train_sorted, y_train_sorted, validation_data=(x_val, y_val), **model_fit_2_params,
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=10)]
        )
              
        y_pred = np.argmax(model.predict(x_test, batch_size=BATCH_SIZE, verbose=0), axis=1)  
        utils.calculate_metrics(y_test, y_pred, acc, re_mi, re_ma, pr_mi, pr_ma, f1_mi, f1_ma)
        if acc[-1] > max(acc):
            best_model_weights = model.get_weights()
            
        print(f"Mean accuracy: {np.mean(acc):.4f}, Current accuracy: {acc[-1]:.4f}")
        
    df_scores = utils.create_df_scores(acc, re_mi, re_ma, pr_mi, pr_ma, f1_mi, f1_ma)
    df_scores.to_csv(f"../data/results/{name}.csv")
    
    model.set_weights(best_model_weights)
    model.save_weights(f"../models/best/{name}.weights.h5")

# Edges

In [ ]:
run_experiments("edges", utils.OrderType.PROBA, name="edges_proba_stl10")

In [ ]:
run_experiments("edges", utils.OrderType.FIXED, name="edges_fixed_stl10")

# Losses

In [ ]:
run_experiments("losses", utils.OrderType.PROBA, name="losses_proba_stl10")

In [ ]:
run_experiments("losses", utils.OrderType.FIXED, name="losses_fixed_stl10")

# Random

In [ ]:
model_fit_3_params = {
    "epochs": 500,
    "batch_size": BATCH_SIZE,
    "shuffle": True,
    "verbose": 0,
}

In [ ]:
def run_experiments_random(name: str):    
    acc, re_mi, re_ma, pr_mi, pr_ma, f1_mi, f1_ma = [], [], [], [], [], [], []
    best_model_weights = model.get_weights()
    
    for _ in tqdm(range(N_TRIALS)):
        model.set_weights(model_weights)
    
        model.fit(
            x_train_sorted, y_train_sorted, validation_data=(x_val, y_val), **model_fit_3_params,
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=35)]
        )
    
        y_pred = np.argmax(model.predict(x_test, batch_size=BATCH_SIZE, verbose=0), axis=1)  
        utils.calculate_metrics(y_test, y_pred, acc, re_mi, re_ma, pr_mi, pr_ma, f1_mi, f1_ma)
        if acc[-1] > max(acc):
            best_model_weights = model.get_weights()
            
        print(f"Mean accuracy: {np.mean(acc):.4f}, Current accuracy: {acc[-1]:.4f}")
        
    df_scores = utils.create_df_scores(acc, re_mi, re_ma, pr_mi, pr_ma, f1_mi, f1_ma)
    df_scores.to_csv(f"../data/results/{name}.csv")

    model.set_weights(best_model_weights)
    model.save_weights(f"../models/best/{name}.weights.h5")

In [ ]:
# with open('RANDOM.txt', 'w') as f:
#     for score in model_scores_random:
#         f.write(f"{score}\n")

In [ ]:
# model_scores_random

In [ ]:
# my_dict = {
#     'model_scores_random': model_scores_random,
#     'model_scores_proba_edges': model_scores_proba_edges,
#     'model_scores_fixed_edges': model_scores_fixed_edges,
#     'model_scores_proba_loss': model_scores_proba_loss,
#     'model_scores_fixed_loss': model_scores_fixed_loss,
# }
# 
# fig, ax = plt.subplots(figsize=(12, 4))
# ax.boxplot(my_dict.values())
# ax.set_xticklabels(my_dict.keys())
# plt.ylim([0.725, 0.865])